In [1]:
import pandas as pd
import os

# --- KONFIGURASI PATH ---
# Mengarah ke folder data/raw di luar folder notebooks
raw_data_path = '../data/raw/data_mentah.csv' 
output_path = '../data/processed/data_bersih.csv'

# 1. LOAD DATA
# Kita baca file CSV. header=1 biasanya baris judul kolom di data PIHPS
try:
    df = pd.read_csv(raw_data_path)
    print("✅ Data berhasil dibaca!")
except FileNotFoundError:
    print("❌ File tidak ditemukan! Pastikan nama file 'data_mentah.csv' ada di folder data/raw/")

# 2. FILTER KOMODITAS (Ambil 3 saja biar fokus)
target_items = [
    'Beras Kualitas Medium I', 
    'Bawang Merah Ukuran Sedang', 
    'Cabai Merah Keriting',
    'Minyak Goreng Kemasan Bermerk 1'    
]
# Filter hanya baris yang nama komoditasnya ada di list target
df_clean = df[df['Komoditas (Rp)'].isin(target_items)].copy()

# 3. BERSIHKAN STRUKTUR (MELT)
# Hapus kolom 'No'
if 'No' in df_clean.columns:
    df_clean = df_clean.drop(columns=['No'])

# Rename kolom Komoditas
df_clean = df_clean.rename(columns={'Komoditas (Rp)': 'Item'})

# Transformasi dari Melebar ke Memanjang (Wajib untuk Time Series)
date_cols = [c for c in df_clean.columns if c != 'Item']
df_melt = df_clean.melt(id_vars=['Item'], value_vars=date_cols, var_name='Tanggal', value_name='Harga')

# 4. FORMAT TIPE DATA
# Bersihkan format Harga (Hapus Rp, titik, koma)
def clean_currency(x):
    if isinstance(x, str):
        # Hapus karakter non-angka
        return x.replace(',', '').replace('.', '').replace('Rp', '').replace(' ', '').strip()
    return x

df_melt['Harga'] = df_melt['Harga'].apply(clean_currency)
df_melt['Harga'] = pd.to_numeric(df_melt['Harga'], errors='coerce')

# Bersihkan format Tanggal (Hapus spasi aneh misal "01/ 01/ 2024")
df_melt['Tanggal'] = df_melt['Tanggal'].str.replace(' ', '')
df_melt['Tanggal'] = pd.to_datetime(df_melt['Tanggal'], format='%d/%m/%Y', errors='coerce')

# 5. SIMPAN HASIL
# Hapus baris yang kosong (NaN)
df_final = df_melt.dropna().sort_values(by=['Item', 'Tanggal'])

# Pastikan folder output ada
os.makedirs('../data/processed', exist_ok=True)

# Simpan ke CSV
df_final.to_csv(output_path, index=False)

print("\n🎉 CLEANING SELESAI!")
print(f"Data bersih disimpan di: {output_path}")
print("Contoh Data:")
print(df_final.head())

✅ Data berhasil dibaca!

🎉 CLEANING SELESAI!
Data bersih disimpan di: ../data/processed/data_bersih.csv
Contoh Data:
                          Item    Tanggal  Harga
1   Bawang Merah Ukuran Sedang 2024-01-01  40000
5   Bawang Merah Ukuran Sedang 2024-01-02  41250
9   Bawang Merah Ukuran Sedang 2024-01-03  41600
13  Bawang Merah Ukuran Sedang 2024-01-04  42000
17  Bawang Merah Ukuran Sedang 2024-01-05  42050
